In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

2022-05-23 13:42:41.625206: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-23 13:42:41.625272: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
class WindowedDatasetGenerator(DatasetGenerator):
    def __init__(
            self,
            train_df: pd.DataFrame,
            validation_df: pd.DataFrame,
            test_df: pd.DataFrame,
            batch_size: int,
            input_width: int,
            label_width: int,
            shift: int,
            label_columns: Optional[List[str]]=None,
    ):
        super(WindowedDatasetGenerator, self).__init__(
            train_df=train_df, validation_df=validation_df, test_df=test_df, batch_size=batch_size
        )
        """
        Example:
            `input_width` = 24 and `shift`=`label_width`=1 --> make a 1 hour prediction based on last 24.
        """
        # Store the raw data.
        self.train_df = train_df
        self.validation_df = validation_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}

        self.column_indices = {name: i for i, name in enumerate(self.train_df.columns)}

        # Compute window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift
        self.total_window_size = input_width + shift

        # Time-based indexing.
        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]
        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def make_dataset(self, df: pd.DataFrame) -> tf.data.Dataset:
        """
        :param df:
        :return: (features, labels): ([batch_size, input_width (time), num_features],
                                        [batch_size, label_width (time), num_labels])
        """
        data = np.array(df, dtype=np.float32)
        dataset = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=self.batch_size,
        )
        return dataset.map(self._split_window)

    def _split_window(self, features: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack([labels[:, :, self.column_indices[name]] for name in self.label_columns], axis=-1)

            # TODO(JP): rm?
            # inputs = tf.stack([inputs[:, :, 1:]], axis=-1)
            inputs = inputs[:, :, 1:]

        # Slicing does not preserve static shape information -> set the
        # shapes manually to make `tf.data.Dataset`s easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels

NameError: name 'DatasetGenerator' is not defined

In [74]:
# train_data_filepath = "/mnt/c/Users/JPhillips/ldz/data/fake_lagged/train.csv"
train_data_filepath = "/mnt/c/Users/JPhillips/ldz/data/processed/train.csv"
train_df = pd.read_csv(train_data_filepath)
train_df.head()

,demand,temp_1,temp_night_1,wind_1,wind_night_1,ssrd_ratio_1,temp_2,temp_night_2,wind_2,wind_night_2,...,ssrd_ratio_12,temp_13,temp_night_13,wind_13,wind_night_13,ssrd_ratio_13,is_weekend,sin_yday,cos_yday,demand_lag1
0,248.551330,0.326607,0.313105,0.164659,0.060791,0.398739,0.305283,0.312067,0.139191,0.060370,...,0.610487,0.250907,0.292878,0.154286,0.045850,0.440716,0,0.000000,1.000000,248.551330
1,242.671902,0.332177,0.293611,0.232767,0.058776,0.790528,0.340104,0.300784,0.189118,0.042210,...,0.753556,0.305485,0.297784,0.217911,0.063559,0.571132,0,0.017166,0.999853,248.551330
2,251.157507,0.279943,0.285802,0.170879,0.076830,0.706229,0.292363,0.292311,0.196887,0.098099,...,0.783674,0.222003,0.262165,0.107576,0.031972,0.898837,1,0.034328,0.999411,242.671902
3,260.073426,0.290286,0.307029,0.271629,0.098776,0.520350,0.308587,0.318814,0.279771,0.112688,...,0.283465,0.274735,0.294953,0.152281,0.053421,0.376753,1,0.051479,0.998674,251.157507
4,277.125461,0.290138,0.284401,0.321351,0.085537,0.747497,0.307230,0.294940,0.250564,0.060241,...,0.716223,0.272833,0.272748,0.269372,0.064238,0.603583,0,0.068615,0.997643,260.073426


In [75]:
train_data = np.array(train_df)

In [76]:
# Work out the label column indices.
label_columns = ["demand"]
if label_columns is not None:
    label_columns_indices = {name: i for i, name in enumerate(label_columns)}

column_indices = {name: i for i, name in enumerate(train_df.columns)}

# Compute window parameters.
input_width = 7
label_width = 1
shift = 0
total_window_size = input_width + shift

# Time-based indexing.
input_slice = slice(0, input_width)
input_indices = np.arange(total_window_size)[input_slice]
label_start = total_window_size - label_width
labels_slice = slice(label_start, None)
label_indices = np.arange(total_window_size)[labels_slice]

train_ds = tf.keras.utils.timeseries_dataset_from_array(
    data=train_data,
    targets=None,
    sequence_length=total_window_size,
    sequence_stride=1,
    # shuffle=True,
    shuffle=False,
    batch_size=1,
)

In [77]:
features = next(iter(train_ds))
features.shape

TensorShape([1, 7, 70])

In [78]:
inputs = features[:, input_slice, :]
labels = features[:, labels_slice, :]

inputs.shape, labels.shape

(TensorShape([1, 7, 70]), TensorShape([1, 1, 70]))

In [79]:
if label_columns is not None:
    labels = tf.stack([labels[:, :, column_indices[name]] for name in label_columns], axis=-1)

    # TODO(JP): rm?
    # inputs = tf.stack([inputs[:, :, 1:]], axis=-1)
    # inputs = inputs[:, :, 1:]
    
inputs.shape, labels.shape

(TensorShape([1, 7, 70]), TensorShape([1, 1, 1]))

In [80]:
inputs2 = inputs[:, :, 1:]
inputs2

<tf.Tensor: shape=(1, 7, 69), dtype=float64, numpy=
array([[[3.26607012e-01, 3.13104758e-01, 1.64659122e-01, 6.07908666e-02,
         3.98738572e-01, 3.05283032e-01, 3.12067293e-01, 1.39190752e-01,
         6.03704164e-02, 3.85568732e-01, 3.22601636e-01, 3.25146956e-01,
         1.39560933e-01, 5.93855484e-02, 3.97239113e-01, 3.12259577e-01,
         3.16571303e-01, 5.98022695e-02, 2.58518977e-02, 4.54233460e-01,
         3.26607012e-01, 3.13104758e-01, 1.64659122e-01, 6.07908666e-02,
         3.98738572e-01, 2.58706830e-01, 2.89557019e-01, 1.01665807e-01,
         3.13049683e-02, 6.01909318e-01, 3.34157149e-01, 3.24901441e-01,
         1.02329531e-01, 4.34028700e-02, 4.68611800e-01, 3.49165085e-01,
         3.22623363e-01, 1.81413383e-01, 5.66615702e-02, 3.72813386e-01,
         3.40939237e-01, 3.21786873e-01, 2.26827505e-01, 7.36065173e-02,
         3.82485145e-01, 3.12475761e-01, 3.13656624e-01, 2.71210661e-01,
         8.84342879e-02, 3.08816996e-01, 2.79961283e-01, 3.01154900e-01,

In [81]:
# Slicing does not preserve static shape information -> set the
# shapes manually to make `tf.data.Dataset`s easier to inspect.
inputs.set_shape([None, input_width, None])
labels.set_shape([None, label_width, None])

In [82]:
features

<tf.Tensor: shape=(1, 7, 70), dtype=float64, numpy=
array([[[2.48551330e+02, 3.26607012e-01, 3.13104758e-01, 1.64659122e-01,
         6.07908666e-02, 3.98738572e-01, 3.05283032e-01, 3.12067293e-01,
         1.39190752e-01, 6.03704164e-02, 3.85568732e-01, 3.22601636e-01,
         3.25146956e-01, 1.39560933e-01, 5.93855484e-02, 3.97239113e-01,
         3.12259577e-01, 3.16571303e-01, 5.98022695e-02, 2.58518977e-02,
         4.54233460e-01, 3.26607012e-01, 3.13104758e-01, 1.64659122e-01,
         6.07908666e-02, 3.98738572e-01, 2.58706830e-01, 2.89557019e-01,
         1.01665807e-01, 3.13049683e-02, 6.01909318e-01, 3.34157149e-01,
         3.24901441e-01, 1.02329531e-01, 4.34028700e-02, 4.68611800e-01,
         3.49165085e-01, 3.22623363e-01, 1.81413383e-01, 5.66615702e-02,
         3.72813386e-01, 3.40939237e-01, 3.21786873e-01, 2.26827505e-01,
         7.36065173e-02, 3.82485145e-01, 3.12475761e-01, 3.13656624e-01,
         2.71210661e-01, 8.84342879e-02, 3.08816996e-01, 2.79961283e-01,

In [83]:
inputs

<tf.Tensor: shape=(1, 7, 70), dtype=float64, numpy=
array([[[2.48551330e+02, 3.26607012e-01, 3.13104758e-01, 1.64659122e-01,
         6.07908666e-02, 3.98738572e-01, 3.05283032e-01, 3.12067293e-01,
         1.39190752e-01, 6.03704164e-02, 3.85568732e-01, 3.22601636e-01,
         3.25146956e-01, 1.39560933e-01, 5.93855484e-02, 3.97239113e-01,
         3.12259577e-01, 3.16571303e-01, 5.98022695e-02, 2.58518977e-02,
         4.54233460e-01, 3.26607012e-01, 3.13104758e-01, 1.64659122e-01,
         6.07908666e-02, 3.98738572e-01, 2.58706830e-01, 2.89557019e-01,
         1.01665807e-01, 3.13049683e-02, 6.01909318e-01, 3.34157149e-01,
         3.24901441e-01, 1.02329531e-01, 4.34028700e-02, 4.68611800e-01,
         3.49165085e-01, 3.22623363e-01, 1.81413383e-01, 5.66615702e-02,
         3.72813386e-01, 3.40939237e-01, 3.21786873e-01, 2.26827505e-01,
         7.36065173e-02, 3.82485145e-01, 3.12475761e-01, 3.13656624e-01,
         2.71210661e-01, 8.84342879e-02, 3.08816996e-01, 2.79961283e-01,

In [84]:
labels

<tf.Tensor: shape=(1, 1, 1), dtype=float64, numpy=array([[[268.9156953]]])>

In [ ]:
# Not getting the features at timestep t in the inputs. Only t-1.

In [ ]:
# How to get it:
# 1. Lag the label.
# 2. Offset = 0.
# 3. `inputs = inputs[:, :, 1:]`, like the labels